In [1]:
import pandas as pd
import gzip
import roman

In [7]:
OLIGO_TABLE_PATH = '/home/shian/CRISPEY3_fraserserver/tables/oligos_nonuniq_crispey3_GG_9bp_OLIGO_with_seq_primers.txt'
oli_info = pd.read_csv(OLIGO_TABLE_PATH,'\t')
af_inf = pd.read_csv('./all_variants.vcf', sep= '\t', header = 59)

/opt/modules/pkgs/anaconda/4.8/envs/CRISPEY_GxE/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_111449/798374678.py:2: DtypeWarning: Columns (24,25,26,27,28,29,37) have mixed types. Specify dtype option on import or set low_memory=False.
  oli_info = pd.read_csv(OLIGO_TABLE_PATH,'\t')


In [3]:
unique_vars = oli_info[['chrom','SNP_chr_pos','ALT']].drop_duplicates()

In [4]:
#reading through the full 1011 yeast genomes gvcf and extracting
#the variants in our library, and converting the chromosome format.
count =0 
rows = []
with gzip.open('1011Matrix.gvcf.gz', 'rt') as g:
    for line in g:
       
        if line.startswith('##'):
            continue
        elif line.startswith('#CHROM'):
            columns = line.strip().split('\t')
        else:
            info = line.strip().split('\t')
            chrom = roman.toRoman(int(info[0].split('e')[1]))
            pos = int(info[1])
            if pos in set(unique_vars[unique_vars['chrom']==chrom]['SNP_chr_pos'].tolist()):
                rows.append([chrom]+info[1:])
        count+=1
        if not count%10000000:
            print(count)

In [5]:
library_variant_genotypes = pd.DataFrame(rows, columns = columns)
library_variant_genotypes['POS'] = library_variant_genotypes['POS'].astype(int)
library_variant_genotypes['QUAL'] = library_variant_genotypes['QUAL'].astype(float)
library_variant_genotypes['ALT_strain'] = library_variant_genotypes['ALT'].iloc[:,1]
library_variant_genotypes['ALT_allele'] = library_variant_genotypes['ALT'].iloc[:,0]
library_variant_genotypes.drop('ALT', axis = 1 ,inplace= True)
library_variant_genotypes['ALT'] = library_variant_genotypes['ALT_allele']
library_variant_genotypes.drop('ALT_allele', axis = 1, inplace = True)

In [15]:
library_variant_genotypes[(library_variant_genotypes['#CHROM']=='I')&(library_variant_genotypes['POS']==195720)]

,#CHROM,POS,ID,REF,QUAL,FILTER,INFO,FORMAT,AAA,AAB,...,SACE_YDH,SACE_YDI,SACE_YDJ,SACE_YDK,SACE_YDL,SACE_YDM,SACE_YDN,SACE_YDO,ALT_strain,ALT
182,I,195720,.,T,226212.29,.,"AC=26,22;AF=0.013,0.011;AN=2000;BaseQRankSum=0...",GT:AD:DP:GQ:PGT:PID:PL,"0/0:210,0,0:210:99:.:.:0,120,1800,120,1800,1800","0/0:188,0,0:188:99:.:.:0,120,1800,120,1800,1800",...,"0/0:308,0,0:308:99:.:.:0,120,1800,120,1800,1800","1/1:0,150,0:150:99:.:.:5738,447,0,5738,447,5738","0/0:358,0,0:358:99:.:.:0,120,1800,120,1800,1800","0/0:79,0,0:79:99:.:.:0,119,1800,119,1800,1800","0/0:132,0,0:132:99:.:.:0,120,1800,120,1800,1800","0/0:64,0,0:64:99:.:.:0,120,1800,120,1800,1800","0/0:313,0,0:313:99:.:.:0,120,1800,120,1800,1800","0/0:133,0,0:133:99:.:.:0,120,1800,120,1800,1800","0/0:197,0,0:197:99:.:.:0,120,1800,120,1800,1800","A,C"


In [17]:
library1011_info_var_id = library_variant_genotypes.merge(af_inf, 'inner', on = ['#CHROM','POS', 'REF','QUAL','FILTER'])

In [18]:
library1011_info_var_id['var_id']= library1011_info_var_id['ID_y']

In [19]:
library1011_info_var_id.drop(['ID_x','ID_y', 'FORMAT_y','FILTER'], axis = 1, inplace = True)

In [20]:
library1011_info_var_id[library1011_info_var_id['var_id']=='GXE_00446']

,#CHROM,POS,REF,QUAL,INFO_x,FORMAT_x,AAA,AAB,AAC,AAD,...,SACE_YDK,SACE_YDL,SACE_YDM,SACE_YDN,SACE_YDO,ALT_strain,ALT_x,ALT_y,INFO_y,var_id
187,I,195720,T,226212.29,"AC=26,22;AF=0.013,0.011;AN=2000;BaseQRankSum=0...",GT:AD:DP:GQ:PGT:PID:PL,"0/0:210,0,0:210:99:.:.:0,120,1800,120,1800,1800","0/0:188,0,0:188:99:.:.:0,120,1800,120,1800,1800","0/0:270,0,0:270:99:.:.:0,120,1800,120,1800,1800","0/0:213,0,0:213:99:.:.:0,120,1800,120,1800,1800",...,"0/0:79,0,0:79:99:.:.:0,119,1800,119,1800,1800","0/0:132,0,0:132:99:.:.:0,120,1800,120,1800,1800","0/0:64,0,0:64:99:.:.:0,120,1800,120,1800,1800","0/0:313,0,0:313:99:.:.:0,120,1800,120,1800,1800","0/0:133,0,0:133:99:.:.:0,120,1800,120,1800,1800","0/0:197,0,0:197:99:.:.:0,120,1800,120,1800,1800","A,C",C,AC=22;AF=0.011;AN=2000;BaseQRankSum=0.878;Clip...,GXE_00446


In [21]:
library1011_info_var_id.to_csv('1011_genome_info_var_ids.tsv', sep='\t')